In [1]:
from nsetools import Nse
from array import array
from time import sleep
import datetime
import xlsxwriter 
from xlsxwriter.utility import xl_range

In [2]:
def colnum_string(n):
    string = ""
    while n > 0:
        n, remainder = divmod(n - 1, 26)
        string = chr(65 + remainder) + string
    return string

nse = Nse()

Listofstocks = ['ADANIPORTS', 'ASIANPAINT', 'AXISBANK', 'BAJAJ-AUTO', 'BAJFINANCE', 'BAJAJFINSV', 'BPCL', 'BHARTIARTL', 'BRITANNIA', 'CIPLA', 'COALINDIA', 'DRREDDY', 'EICHERMOT', 'GAIL', 'GRASIM', 'HCLTECH', 'HDFCBANK','HEROMOTOCO', 'HINDALCO', 'HINDUNILVR', 'HDFC', 'ICICIBANK', 'ITC', 'IOC', 'INDUSINDBK', 'INFY', 'JSWSTEEL', 'KOTAKBANK', 'LT', 'M&M', 'MARUTI', 'NTPC', 'NESTLEIND', 'ONGC', 'POWERGRID', 'RELIANCE', 'SHREECEM', 'SBIN','SUNPHARMA', 'TCS', 'TATAMOTORS', 'TATASTEEL', 'TECHM']


In [5]:
workbook = xlsxwriter.Workbook('Nse_24thMay-data.xlsx') 
worksheet = workbook.add_worksheet('')
j=0
k=0
o=0
st=0
y=[]

#Key Variableto update
max_counter=120
#Sp=10

Repeat_round = 7
sn=1
print (datetime.datetime.now())

for o in range (0,len(Listofstocks)):
    worksheet.write(0,o+0+st,'SrNo')
    worksheet.write(0,o+1+st,'Symbol')
    worksheet.write(0,o+2+st,'totalBuyQuantity')
    worksheet.write(0,o+3+st,'totalSellQuantity')
    worksheet.write(0,o+4+st,'Buy - Sell')
    worksheet.write(0,o+5+st,'Buy - Sell % Force')
    worksheet.write(0,o+6+st,'Time')
    st=st+Repeat_round

worksheet2 = workbook.add_worksheet('Summary')
worksheet2.write(0,0,'Stock Name')
worksheet2.write(0,1,'Buy-Sell Force')
worksheet2.write(0,2,'Volatity')
worksheet2.write(0,3,'CMP')
worksheet2.write(0,4,'Day Open')
worksheet2.write(0,5,'Direction')
worksheet2.write(0,6,'Time')


2021-05-25 09:53:42.556261


0

In [6]:
while True:
    stock_1 = []
    s = 0
    
    for i in range(0,len(Listofstocks)):
        print(i)
        q = nse.get_quote(Listofstocks[i])

        if q['totalBuyQuantity'] is None:
            q['totalBuyQuantity']=0
        if q['totalSellQuantity'] is None:
            q['totalSellQuantity']=0
            
        if q['totalBuyQuantity']==0 and q['totalSellQuantity']==0 :
            BUY_Perc = None
        else:
            BUY_Perc= (q['totalBuyQuantity']-q['totalSellQuantity'])/(q['totalBuyQuantity']+q['totalSellQuantity'])

        x=[q['symbol'],q['totalBuyQuantity'],q['totalSellQuantity'],q['totalBuyQuantity']-q['totalSellQuantity'],BUY_Perc]

        worksheet.write(j+1,i+0+s,j+1)
        worksheet.write(j+1,i+1+s,x[0])
        worksheet.write(j+1,i+2+s,x[1])
        worksheet.write(j+1,i+3+s,x[2])
        worksheet.write(j+1,i+4+s,x[3])
        worksheet.write(j+1,i+5+s,x[4])
        worksheet.write(j+1,i+6+s,"%s:%s:%s" % (datetime.datetime.now().hour, datetime.datetime.now().minute,datetime.datetime.now().second))
    
        if j == max_counter-1:
            worksheet2.write(sn,0,q['symbol'])
            worksheet2.write(sn,3,q['lastPrice'])
            worksheet2.write(sn,4,q['open'])
            sn=sn+1
    
        if i != len(Listofstocks):
            #this is the last count and last round trip. all the numbers are plugged. Last final straw pending
            if j == max_counter-1:
                sb=0
                # print (j)

                for si in range(0,len(Listofstocks)):
                    cell_range = xl_range(1,si+5+sb,j+1,si+5+sb)
                    #print (cell_range)
                    formula = '=Average(%s)' % cell_range
                    worksheet.write_formula(j+2,si+5+sb,formula)
                    sb=sb+Repeat_round
                    

                sb=0              
                # print (si)

            
                for sm in range(0,len(Listofstocks)):
                    y1 = ('=Sheet1!' ,colnum_string(6+sb),j+2+1)
                    Ref1 = ("%s%s%d" % y1)
                    #  print(Ref1)
                    worksheet2.write_formula(sm+1,1,Ref1)

                    #=IF(B2>0,"Buy","Sell")
                    y2 = ('=IF(','B',sm+2, '>0,"BUY","SELL")')
                    Ref2 = ("%s%s%d%s" % y2)
                    #print (Ref2)
                    worksheet2.write(sm+1,5,Ref2)
                    worksheet2.write(sm+1,6,"%s:%s:%s" % (datetime.datetime.now().hour, datetime.datetime.now().minute,datetime.datetime.now().second))
                    sb=sb+Repeat_round+1


                cell_range1 = xl_range(1,5,sm+1,5)
                y3= ('=COUNTIF(',cell_range1,',"BUY")')
                Ref3 = ("%s%s%s" % y3)
            #print (Ref3)
                worksheet2.write(0,8,"Total Buy")
                worksheet2.write_formula(0,9,Ref3)

                cell_range1 = xl_range(1,5,sm+1,5)
                y4= ('=COUNTIF(',cell_range1,',"SELL")')
                Ref4 = ("%s%s%s" % y4)
            #print (Ref4)
                worksheet2.write(1,8,"Total Sell")
                worksheet2.write_formula(1,9,Ref4)
            s=s+Repeat_round


    k= k+ len(Listofstocks)-1
    j=j+1
    #sleep(0)
    if j==max_counter:
        #worksheet_Sum.write(0,0,'test')
        break
    
print (datetime.datetime.now())
workbook.close()

0
1
2


URLError: <urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>